In [1]:
# arn:aws:s3:::export-test-tmuth
# https://export-test-tmuth.s3.amazonaws.com/test1/
# s3://export-test-tmuth/test1/
from smart_open import open

In [13]:
from smart_open import s3
# we use workers=1 for reproducibility; you should use as many workers as you have cores
bucket = 'export-test-tmuth'
prefix = 'test1'
for key, content in s3.iter_bucket(bucket, prefix=prefix, accept_key=lambda key: '/t' in key, workers=1, key_limit=3):
    print(key, round(len(content) / 2**20))

test1/test.json 0
test1/test.out 1
test1/test1.txt 0


In [18]:
import boto3
from pprint import pprint
from smart_open import open
# tp = {'min_part_size': 5 * 1024**2}
lots_of_data="Hello world"
resource = boto3.resource('s3') 
with open('s3://export-test-tmuth/test1/test1.txt', 'w') as fout:
    # fout.write(lots_of_data)
    print(lots_of_data,file=fout)
    print(fout.tell())
    boto3_object = fout.to_boto3(resource)
    pprint(boto3_object)


12
s3.Object(bucket_name='export-test-tmuth', key='test1/test1.txt')


In [15]:
import boto3

s3 = boto3.resource('s3')
copy_source = {
    'Bucket': 'export-test-tmuth',
    'Key': 'test1/test1.txt'
}
s3.meta.client.copy(copy_source, 'export-test-tmuth', 'test1/test2.txt')


In [17]:
# delete = s3.delete_object(Bucket='export-test-tmuth', Key='test1/test2.txt')
s3.Object('export-test-tmuth', 'test1/test2.txt').delete()

{'ResponseMetadata': {'RequestId': 'M9SWZ9HR943RRR9C',
  'HostId': '3l4GTf1sh3vyoU0QS77EbA23FfiyH9ebDEmImykbyA/Hu4/bLDeDaiz2ZnKFQNc9I2bNXnudIGU=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': '3l4GTf1sh3vyoU0QS77EbA23FfiyH9ebDEmImykbyA/Hu4/bLDeDaiz2ZnKFQNc9I2bNXnudIGU=',
   'x-amz-request-id': 'M9SWZ9HR943RRR9C',
   'date': 'Fri, 04 Nov 2022 12:15:50 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

In [19]:
def split_s3_path(s3_path):
    path_parts=s3_path.replace("s3://","").split("/")
    bucket=path_parts.pop(0)
    key="/".join(path_parts)
    return bucket, key

bucket, key = split_s3_path("s3://my-bucket/some_folder/another_folder/my_file.txt")
print(bucket)
print(key)

my-bucket
some_folder/another_folder/my_file.txt
